In [1]:
from graph.src.triplet_extraction import init_gpt, init_vncorenlp
from graph.src.db import init_sqlite, init_neo4j, delete_all

process_conn, process_cursor = init_sqlite(r"E:\Github\uit_chatbot\graph\process_law.db")
law_conn, law_cursor = init_sqlite(r"E:\Github\uit_chatbot\graph\GTVT_law.db")
gpt_client = init_gpt()
neo4j_session = init_neo4j(
        uri="neo4j://127.0.0.1:7687",
        username="neo4j",
        password="1234567890",
        db_name="neo4j",
    )

In [2]:
vncorenlp_client = init_vncorenlp(r"E:\Github\uit_chatbot\graph\VnCoreNLP-1.2")

In [3]:
import phonlp
phoNLP_model = phonlp.load(save_dir=r"E:\Github\uit_chatbot\graph\phonlp")

Loading model from: E:\Github\uit_chatbot\graph\phonlp/phonlp.pt


In [51]:
def parsing_result(annotation):
    import pandas as pd
    words = annotation[0][0]
    pos_tags_nested = annotation[1][0]
    ner_tags = annotation[2][0]
    dep_tags_nested = annotation[3][0]
    # Create the 'id' list
    ids = list(range(1, len(words) + 1))

    # Flatten the POS tags
    pos_tags = [tag[0] for tag in pos_tags_nested]

    # Split the dependency tags
    heads = [dep[0] for dep in dep_tags_nested]
    deprels = [dep[1] for dep in dep_tags_nested]

    data = {
        'id': ids,
        'word': words,
        'pos': pos_tags,
        # 'ner': ner_tags,
        'head': heads,
        'deprel': deprels
    }

    return pd.DataFrame(data)

In [57]:
from graph.src.triplet_extraction import clean_text
from graph.src.db import extract_from_sqlite

rows = extract_from_sqlite(law_cursor, "89cdd5f3440d0fd24279d51d996983123336cb3022163dd5fe9866f153ccfd05", True)
sentence = ""
title = ""
for r in rows:
    title += r['title'] + " "
    if not (r['title'].strip().startswith("Chương") or r['title'].strip().startswith("Điều")):
        sentence += r['content'] + "\n"

so_hieu = r['so_hieu']
ID = r['id']
print(ID)
print(so_hieu + " " + title)
print(clean_text(sentence))

89cdd5f3440d0fd24279d51d996983123336cb3022163dd5fe9866f153ccfd05
16/2022/TT-BGTVT Điều 9 Khoản 7 Điểm b 
đối với thẩm định tài liệu hướng dẫn, hồ sơ bao gồm: 01 bản sao điện tử có giá trị pháp lý (đối với trường hợp nộp thông qua hệ thống dịch vụ công trực tuyến) hoặc 03 bản chính (đối với trường hợp nộp trực tiếp hoặc qua hệ thống bưu chính hoặc hình thức phù hợp khác) tài liệu hướng dẫn.”


In [20]:
from graph.src.db import extract_random_from_sqlite

rows = extract_random_from_sqlite(process_cursor)
sentence = rows['content']
so_hieu = rows['so_hieu']
document_id = rows['id']

print(document_id)
print(so_hieu)
print(sentence)
print()

764ee4312a8206628f62ba0285eeb51880713853f374dd40e65b3e82aacc2a08
36/2024/QH15
Nhiệm vụ của cảnh sát giao thông khi thực hiện tuần tra, kiểm soát bao gồm bảo đảm an ninh, trật tự theo quy định của pháp luật



In [ ]:
from graph.src.triplet_extraction import process_sentence

result = process_sentence(sentence, vncorenlp_client, True)

In [21]:
text = sentence
segmented_text = vncorenlp_client.word_segment(text)
annotation = phoNLP_model.annotate(text=segmented_text[0])
df = parsing_result(annotation)
df.style.hide(axis="index")

100%|██████████| 1/1 [00:00<00:00, 11.54it/s]


id,word,pos,head,deprel
1,Nhiệm_vụ,N,10,sub
2,của,E,1,nmod
3,cảnh_sát,N,2,pob
4,giao_thông,N,3,nmod
5,khi,N,1,tmp
6,thực_hiện,V,5,nmod
7,tuần_tra,V,6,vmod
8,",",CH,7,punct
9,kiểm_soát,V,7,vmod
10,bao_gồm,V,0,root


In [54]:
text = "Chủ đầu tư có trách nhiệm bảo trì đoạn đường vừa thi công, vừa khai thác sử dụng trong thời gian bảo hành công trình"
segmented_text = vncorenlp_client.word_segment(text)
annotation = phoNLP_model.annotate(text=segmented_text[0])
df = parsing_result(annotation)
print(df.to_string(index=False))

100%|██████████| 1/1 [00:00<00:00, 12.09it/s]

 id        word pos head deprel
  1         Chủ   N    3    sub
  2      đầu_tư   V    1   nmod
  3          có   V    0   root
  4 trách_nhiệm   N    3    dob
  5     bảo_trì   V    4   nmod
  6        đoạn  Nc    5    dob
  7       đường   N    6   nmod
  8         vừa   R    9    adv
  9    thi_công   V    6   nmod
 10           ,  CH    9  punct
 11         vừa   R   12    adv
 12   khai_thác   V    6   nmod
 13     sử_dụng   V   12   vmod
 14       trong   E   12    tmp
 15   thời_gian   N   14    pob
 16    bảo_hành   V   15   nmod
 17  công_trình   N   16   vmod


In [ ]:
def triplet_extraction(df):
    data_dict = df.set_index('id').to_dict(orient='index')

    root_id = None

    part1 = []
    part2 = []

    for token_id, info in data_dict.items():
        if not root_id:
            part1.append(token_id)
        else:
            part2.append(token_id)

        if info['deprel'] == "root" and info['pos'].startswith("V"):
            root_id = token_id
            break

    if root_id:
        print("No root found")
        return

    #Handle part 1
    sub_id = next((i for i in part1 if data_dict[i]['deprel'] == 'sub'), None)
    if sub_id:
        print("No sub found")
        return



In [10]:
import os

# Save all dependency relation tags to a file
output_path = r"E:\Github\uit_chatbot\graph\deprel_tags.txt"

with open(output_path, "w", encoding="utf-8") as f:
    for tag in all_deprel_tag:
        f.write(tag + "\n")

full_path = os.path.abspath(output_path)
print(f"Saved {len(all_deprel_tag)} tags to {full_path}")


Saved 24 tags to E:\Github\uit_chatbot\graph\deprel_tags.txt


In [98]:
def process_sentence(df, verbose: bool = True) -> dict:
    data_dict = df.set_index('id').to_dict(orient='index')

    # Triplet extraction
    triplets = []
    concept1_tokens = []
    concept2_tokens = []
    relation_tokens = []

    verb_deprel_tag = ['root', 'vmod', 'nmod', 'x', 'conj', 'prd', 'tpc', 'dep']
    remove_POS_tag = ["R", "CH"]

    for token_id, info in data_dict.items():
        if any(info['pos'].startswith(prefix) for prefix in remove_POS_tag):
            continue

        if (token_id - 1) > 0 and (token_id + 1) <= len(df):
            between_n = (data_dict[token_id - 1]['pos'].startswith("N") and data_dict[token_id + 1]['pos'].startswith("N"))
        else:
            between_n = False
        token = info['word']

        if (info['pos'].startswith("V") and (info['deprel'] in verb_deprel_tag)
                and (info['deprel'] != "nmod" or not between_n) and info['deprel'] != "vmod"):
            if concept2_tokens:
                triplet = (
                    " ".join(concept1_tokens),
                    " ".join(relation_tokens),
                    " ".join(concept2_tokens)
                )
                if triplet not in triplets:  # chỉ thêm nếu chưa tồn tại
                    triplets.append(triplet)

                concept1_tokens = concept2_tokens
                concept2_tokens = []
                relation_tokens = []
            relation_tokens.append(token)
        else:
            if relation_tokens:
                concept2_tokens.append(token)
            else:
                concept1_tokens.append(token)

    if concept1_tokens and relation_tokens and concept2_tokens:
        triplets.append((
            " ".join(concept1_tokens),
            " ".join(relation_tokens),
            " ".join(concept2_tokens)
        ))

    return triplets

rows = extract_random_from_sqlite(process_cursor)
rand_sentence = rows['content']
# rand_sentence = ""
print(rand_sentence)
segmented_text = vncorenlp_client.word_segment(rand_sentence)
annotation = phoNLP_model.annotate(text=segmented_text[0])
df = parsing_result(annotation)
print(df.to_string(index=False))
result = process_sentence(df)
for triplet in result:
    print(triplet)

Chi phí vận hành khai thác công trình đường bộ được xác định theo quy định của pháp luật có liên quan


100%|██████████| 1/1 [00:00<00:00, 12.16it/s]

 id       word pos head deprel
  1    Chi_phí   N    6    sub
  2   vận_hành   V    1   nmod
  3  khai_thác   V    2   vmod
  4 công_trình   N    2   vmod
  5   đường_bộ   N    4   nmod
  6       được   V    0   root
  7   xác_định   V    6   vmod
  8       theo   E    7    mnr
  9   quy_định   V    8   vmod
 10        của   E    9   nmod
 11  pháp_luật   N   10    pob
 12         có   V   11   nmod
 13  liên_quan   V   12   vmod
('Chi_phí', 'vận_hành', 'khai_thác công_trình đường_bộ')
('khai_thác công_trình đường_bộ', 'được', 'xác_định theo quy_định của pháp_luật')
('xác_định theo quy_định của pháp_luật', 'có', 'liên_quan')


In [108]:
def process_sentence(df, verbose: bool = True) -> dict:
    data_dict = df.set_index('id').to_dict(orient='index')

    # === 1. Tiền xử lý: Tìm chủ ngữ (sub) của tất cả các động từ ===
    # Tạo một bản đồ: {verb_id: [subject_id_1, subject_id_2, ...]}
    subjects_of_verbs = {}
    for token_id, info in data_dict.items():
        if info['deprel'] == 'sub':
            head_id = info['head'] # ID của động từ
            if head_id not in subjects_of_verbs:
                subjects_of_verbs[head_id] = []
            subjects_of_verbs[head_id].append(token_id) # ID của chủ ngữ

    # === 2. Trích xuất Triplet ===
    triplets = []
    # Thay đổi: Lưu trữ ID thay vì từ (word)
    concept1_ids = []
    concept2_ids = []
    relation_ids = []

    verb_deprel_tag = ['root', 'vmod', 'nmod', 'x', 'conj', 'prd', 'tpc', 'dep']
    # Thêm 'E' (Giới từ) vào danh sách loại bỏ vì chúng thường là nhiễu
    remove_POS_tag = ["R", "CH", "E"]

    # Hàm trợ giúp để chuyển danh sách ID thành chuỗi
    def ids_to_string(id_list):
        id_list.sort() # Sắp xếp các ID để đảm bảo thứ tự từ là đúng
        return " ".join(data_dict[tid]['word'] for tid in id_list)

    for token_id, info in data_dict.items():
        if any(info['pos'].startswith(prefix) for prefix in remove_POS_tag):
            continue

        if (token_id - 1) > 0 and (token_id + 1) <= len(df):
            between_n = (data_dict[token_id - 1]['pos'].startswith("N") and data_dict[token_id + 1]['pos'].startswith("N"))
        else:
            between_n = False
        token = info['word']

        # --- Quyết định token này là Relation hay Concept ---
        is_relation = False
        if (info['pos'].startswith("V") and (info['deprel'] in verb_deprel_tag)
                and (info['deprel'] != "nmod" or not between_n) and info['deprel'] != "vmod"):

            # Mặc định coi nó là relation
            is_relation = True

            # === 3. LOGIC MỚI: KIỂM TRA ĐỘNG TỪ ROOT ===
            if info['deprel'] == 'root':
                # Lấy danh sách ID chủ ngữ của động từ 'root' này
                subject_ids_for_this_root = subjects_of_verbs.get(token_id, [])

                if not subject_ids_for_this_root:
                    # Động từ 'root' này không có 'sub' (ví dụ: câu mệnh lệnh)
                    # -> Không coi nó là relation, mà là một phần của concept
                    is_relation = False
                else:
                    # Kiểm tra xem concept1 có chứa bất kỳ ID chủ ngữ nào không
                    found_subject_in_concept1 = False
                    for sub_id in subject_ids_for_this_root:
                        if sub_id in concept1_ids:
                            found_subject_in_concept1 = True
                            break

                    if not found_subject_in_concept1:
                        # Chủ ngữ không nằm trong concept1.
                        # -> Không coi nó là relation, mà là một phần của concept
                        is_relation = False
            # === Kết thúc logic mới ===

        # --- Thêm token_id vào đúng danh sách ---
        if is_relation:
            # Đây là một Relation
            if concept2_ids:
                # Nếu đã có concept2, tức là ta vừa kết thúc một triplet cũ
                triplet = (
                    ids_to_string(concept1_ids),
                    ids_to_string(relation_ids),
                    ids_to_string(concept2_ids)
                )
                if triplet not in triplets:
                    triplets.append(triplet)

                # Concept 2 cũ trở thành Concept 1 mới
                concept1_ids = concept2_ids[:] # Phải copy
                concept2_ids = []
                relation_ids = []

            relation_ids.append(token_id)
        else:
            # Đây là một Concept
            if info['deprel'] == "vmod" and len(concept2_ids) == 0:
                relation_ids.append(token_id)
            elif relation_ids:
                concept2_ids.append(token_id)
            else:
                # Nếu chưa có relation, token này thuộc về concept 1
                concept1_ids.append(token_id)

    # Xử lý triplet cuối cùng còn sót lại
    if concept1_ids and relation_ids and concept2_ids:
        triplet = (
            ids_to_string(concept1_ids),
            ids_to_string(relation_ids),
            ids_to_string(concept2_ids)
        )
        if triplet not in triplets:
            triplets.append(triplet)

    return triplets

# rows = extract_random_from_sqlite(process_cursor)
rand_sentence = rows['content']
# rand_sentence = ""
print(rand_sentence)
segmented_text = vncorenlp_client.word_segment(rand_sentence)
annotation = phoNLP_model.annotate(text=segmented_text[0])
df = parsing_result(annotation)
print(df.to_string(index=False))
result = process_sentence(df)
for triplet in result:
    print(triplet)

Chi phí vận hành khai thác công trình đường bộ được xác định theo quy định của pháp luật có liên quan


100%|██████████| 1/1 [00:00<00:00, 12.35it/s]

 id       word pos head deprel
  1    Chi_phí   N    6    sub
  2   vận_hành   V    1   nmod
  3  khai_thác   V    2   vmod
  4 công_trình   N    2   vmod
  5   đường_bộ   N    4   nmod
  6       được   V    0   root
  7   xác_định   V    6   vmod
  8       theo   E    7    mnr
  9   quy_định   V    8   vmod
 10        của   E    9   nmod
 11  pháp_luật   N   10    pob
 12         có   V   11   nmod
 13  liên_quan   V   12   vmod
('Chi_phí', 'vận_hành khai_thác công_trình', 'đường_bộ được xác_định quy_định pháp_luật')


In [163]:
import pandas as pd
from itertools import product

# Đã sửa logic nmod và vmod
def process_sentence(df):
    data_dict = df.set_index('id').to_dict(orient='index')

    if data_dict[1]['deprel'] == 'root':
        return None

    # === 1. Tiền xử lý ===
    subjects_of_verbs = {}
    for token_id, info in data_dict.items():
        if info['deprel'] == 'sub':
            head_id = info['head']
            subjects_of_verbs.setdefault(head_id, []).append(token_id)

    # === 2. Khởi tạo ===
    triplets = []
    concept1_groups = [[]]
    concept2_groups = [[]]
    relation_groups = [[]]

    verb_deprel_tag = ['root', 'vmod', 'nmod', 'x', 'conj', 'prd', 'tpc', 'dep']
    remove_POS_tag = ["R", "CH", "E", "L"]
    coord_POS_tag = ["C"]
    coord_words = {"và", "hoặc", ","}

    def ids_to_string(id_list):
        id_list = sorted(set(id_list))
        return " ".join(data_dict[tid]['word'] for tid in id_list if tid in data_dict)

    def is_coord_word(info):
        return (info["word"].lower() in coord_words or
                any(info['pos'].startswith(prefix) for prefix in coord_POS_tag))

    token_ids = sorted(list(data_dict.keys()))
    token_set = set(token_ids)

    # === 3. Duyệt tokens ===
    last_was_coord = False

    for token_id in token_ids:
        info = data_dict[token_id]

        # Logic bỏ qua token
        if any(info['pos'].startswith(prefix) for prefix in remove_POS_tag) and info['deprel'] != 'root':
            continue

        # Logic từ nối
        if is_coord_word(info):
            last_was_coord = True
            continue

        is_relation = False

        # --- Logic 'vmod' (bắt buộc phải là 'V') ---
        is_vmod_continuation = (
            info['pos'].startswith("V")
            and info['deprel'] == "vmod"
            and any(g for g in relation_groups if g)
        )

        # --- Logic 'relation' khác (bắt buộc phải là 'V') ---
        prev_is_n = (token_id - 1 in token_set and data_dict[token_id - 1]['pos'].startswith("N"))
        next_is_n = (token_id + 1 in token_set and data_dict[token_id + 1]['pos'].startswith("N"))

        is_valid_relation_type = (
            info['pos'].startswith("V")
            and info['deprel'] in verb_deprel_tag
            and info['deprel'] not in ("vmod", "root")
            and (info['deprel'] != "nmod" or (prev_is_n and next_is_n))
        )

        # --- Logic 'root' (Có thể là 'V' hoặc 'R') ---
        is_valid_root = False
        if info['deprel'] == "root":
            is_valid_root = True

        # Quyết định cuối cùng
        if is_vmod_continuation or is_valid_relation_type or is_valid_root:
            is_relation = True
        target_groups = None

        if is_relation:
            # print(token_id) # Bỏ comment để debug
            if any(g for g in concept2_groups if g):
                # Hoàn thành triplet cũ
                for c1g, rg, c2g in product(concept1_groups, relation_groups, concept2_groups):
                    triplet = (ids_to_string(c1g), ids_to_string(rg), ids_to_string(c2g))
                    if triplet not in triplets and c1g and rg and c2g:
                        triplets.append(triplet)

                concept1_groups = concept2_groups[:]
                concept2_groups = [[]]
                relation_groups = [[]]

            target_groups = relation_groups
        else:
            # Nó là một Concept
            if any(g for g in relation_groups if g):
                target_groups = concept2_groups
            else:
                target_groups = concept1_groups

        if last_was_coord:
            # FORK: Tạo nhóm mới
            if target_groups is not None:
                target_groups.append([token_id])
            last_was_coord = False
        else:
            # APPEND: Thêm vào nhóm cuối
            if target_groups is not None:
                if not target_groups or not target_groups[-1]:
                     target_groups.append([token_id])
                else:
                     target_groups[-1].append(token_id)

    # === 6. Kết thúc (Giữ nguyên) ===
    for c1g, rg, c2g in product(concept1_groups, relation_groups, concept2_groups):
        if c1g and rg and c2g:
            triplet = (
                ids_to_string(c1g),
                ids_to_string(rg),
                ids_to_string(c2g)
            )
            if triplet not in triplets:
                triplets.append(triplet)
    return triplets

rows = extract_random_from_sqlite(process_cursor)
rand_sentence = rows['content']
# rand_sentence = ""
print(rand_sentence)
segmented_text = vncorenlp_client.word_segment(rand_sentence)
annotation = phoNLP_model.annotate(text=segmented_text[0])
df = parsing_result(annotation)
print(df.to_string(index=False))
result = process_sentence(df)
if result:
    for triplet in result:
        print(triplet)
else:
    print("There was a problem with the sentence.")

Cơ quan quản lý đường bộ có trách nhiệm thông báo kết quả khắc phục cho cá nhân đã kiến nghị trong phạm vi nhiệm vụ, quyền hạn của mình


100%|██████████| 1/1 [00:00<00:00,  9.64it/s]

 id        word pos head deprel
  1     Cơ_quan   N    4    sub
  2     quản_lý   V    1   nmod
  3    đường_bộ   N    2    dob
  4          có   V    0   root
  5 trách_nhiệm   N    4    dob
  6   thông_báo   V    5   nmod
  7     kết_quả   N    6    dob
  8   khắc_phục   V    7   nmod
  9         cho   E    6    iob
 10     cá_nhân   N    9    pob
 11          đã   R   12    adv
 12   kiến_nghị   V   10   nmod
 13       trong   E   12    loc
 14     phạm_vi   N   13    pob
 15    nhiệm_vụ   N   14   nmod
 16           ,  CH   15  punct
 17   quyền_hạn   N   15   nmod
 18         của   E   15   nmod
 19        mình   P   18    pob
('Cơ_quan', 'quản_lý', 'đường_bộ')
('đường_bộ', 'có', 'trách_nhiệm')
('trách_nhiệm', 'thông_báo', 'kết_quả khắc_phục cá_nhân kiến_nghị phạm_vi nhiệm_vụ quyền_hạn mình')


In [120]:
data_dict = df.set_index('id').to_dict(orient='index')

# === 1. Tiền xử lý: Tìm chủ ngữ (sub) của tất cả các động từ ===
subjects_of_verbs = {}
for token_id, info in data_dict.items():
    if info['deprel'] == 'sub':
        head_id = info['head']
        if head_id not in subjects_of_verbs:
            subjects_of_verbs[head_id] = []
        subjects_of_verbs[head_id].append(token_id)

for subject_id in subjects_of_verbs:
    print(subject_id)
    print(subjects_of_verbs[subject_id])

7
[1]
